This is another notebook for preprocessing but the difference here is that we mapped emojis and emoticons into Arabic text 


In [7]:
#importing libaraires

# Add environment Packages paths to conda
import os, sys, warnings
import pandas as pd
import numpy as np
warnings.simplefilter("ignore")

# Text preprocessing packages
import nltk # Text libarary
# nltk.download('stopwords')
import string # Removing special characters {#, @, ...}
import re # Regex Package
import regex
import emoji
# Corpora is a group presenting multiple collections of text documents. A single collection is called corpus.
from nltk.corpus import stopwords # Stopwords
import arabicstopwords.arabicstopwords as stp #more range of arabic stop words
from nltk.stem.isri import ISRIStemmer
import pyarabic.araby as araby
from tashaphyne.stemming import ArabicLightStemmer

from nltk.stem import SnowballStemmer, WordNetLemmatizer # Stemmer & Lemmatizer
#from gensim.utils import simple_preprocess  # Text ==> List of Tokens

# Text Embedding
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
                                                       
# Modelling
from sklearn.model_selection import train_test_split,KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.svm import SVC
# Saving Model
import pickle

# Visualization Packages
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale=1.3)
%matplotlib inline

In [2]:
#!pip install nltk
pip install Arabic-Stopwords
pip install PyArabic
pip install Tashaphyne
#pip install translate
#pip install transformers

SyntaxError: ignored

In [3]:
pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 197 kB 5.2 MB/s 
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=1553ca02b90849487ae5278978235ec117db5ec00cb9d87a842de590fcaa5358
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe7452ac7d8d83b1930f8a6205c3c9649b24e80f9029fc38
Successfully built emoji


In [4]:
pip install Arabic-Stopwords

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 353 kB 5.3 MB/s 
     |████████████████████████████████| 126 kB 60.1 MB/s 


In [5]:
pip install PyArabic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
pip install Tashaphyne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 251 kB 4.9 MB/s 


In [11]:
df=pd.read_excel('NileUniversity_Emotional.xlsx')
df.head()

FileNotFoundError: ignored

In [ ]:
#deop unnecessary columns
df.drop('ID', axis=1,inplace=True)

## Assessing

In [ ]:
#print the shape of the data set
df.shape

In [ ]:
df.info()

In [ ]:
#check if the data is balanced or not
df.label.value_counts()

In [ ]:
plt.figure(figsize=(8,4))
sns.countplot(x='label', data=df);

The differences between the labels is not large so, we can say that the data is balanced

In [ ]:
#print the number of null values in each column
df.isnull().sum()

In [ ]:
#to get the index of the null value
df[df['text'].isnull() == True]

This label doesn't mean anything because the original text is missing so, we will remove it

In [ ]:
#removing null value
df.drop(8619, axis=0, inplace = True)

#resetting the index of the data
df.reset_index(drop=True, inplace = True)

In [ ]:
#print the number of duplicated values 
df.duplicated().sum()

In [ ]:
#print the rows which are duplicated
df[df['text'].duplicated() == True]

In [ ]:
df[df['text'] == df.iloc[1236]['text']]

In [ ]:
#removing duplicated values
index = df[df['text'].duplicated() == True].index
df.drop(index, axis = 0, inplace = True)

In [ ]:
#are there any stop words in the data?
arabic_stopwords = stopwords.words("arabic")
len(arabic_stopwords)

Note that There are different libraries provide a different numbers of arabic stop words like "nltk" and "arabicstopwords" now we will use nltk's arabic stopwords and we will see the results and if needed we can use "arabicstopwords" library which provide the stopwords and it's conjugations

In [ ]:
df['stop_words'] = df['text'].apply(lambda x: len(set(x.split()) & set(arabic_stopwords)))
df.stop_words.value_counts()

In [ ]:
df['stop_words'].plot(kind= 'hist')

it seems that the data contains alot of stop words and it should be removed

In [ ]:
#to count the number of emojis in the dataset
def emoji_counter(sentence):
    return emoji.emoji_count(sentence)

df['emoji_count'] = df['text'].apply(lambda x: emoji_counter(x))
df.emoji_count.value_counts()[:10]

most of the records doesn't contain emojies but there are about 1910 emoji that we should take care during preprocessing

**extract the emojis from every text**

In [ ]:
emojis_data = df[df['emoji_count'] > 0]

In [ ]:
def extract_emoji(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)
    
    return emoji_list

In [ ]:
emojis_data['emoji'] = emojis_data['text'].apply(lambda x: extract_emoji(x))
emojis_data['emoji'] = emojis_data['emoji'].apply(lambda x: ' '.join([word for word in x ]))

In [ ]:
emojis_data

In [ ]:
#find most repeated emojis
pd.Series(' '.join(emojis_data['emoji']).split()).value_counts()[:10]

Face With Tears of Joy is the most repeated emoji

In [ ]:
df.text.iloc[99]

After Exploring some records we found Diacritics in the dataset and we should put this in our considerations

In [ ]:
df

### Assessing Results

- The data contains null values
- The data contains duplicated values
- The data is balanced and contains 8 classes
- About 20% of the data contains emojis
- More than 85% of the text in each records contains stopwords
- the data contains Diacritics 
- The data contains Punctuations, Disapprobation words and hashtags and other things that we should take care during cleaning and preprocessing
- Face With Tears of Joy is the most repeated emoji

## Cleaning

### Constants and important variables

In [ ]:
emojis = {
    "🙂":"يبتسم",
    "😂":"يضحك",
    "💔":"قلب حزين",
    "🙂":"يبتسم",
    "❤️":"حب",
    "❤":"حب",
    "😍":"حب",
    "😭":"يبكي",
    "😢":"حزن",
    "😔":"حزن",
    "♥":"حب",
    "💜":"حب",
    "😅":"يضحك",
    "🙁":"حزين",
    "💕":"حب",
    "💙":"حب",
    "😞":"حزين",
    "😊":"سعادة",
    "👏":"يصفق",
    "👌":"احسنت",
    "😴":"ينام",
    "😀":"يضحك",
    "😌":"حزين",
    "🌹":"وردة",
    "🙈":"حب",
    "😄":"يضحك",
    "😐":"محايد",
    "✌":"منتصر",
    "✨":"نجمه",
    "🤔":"تفكير",
    "😏":"يستهزء",
    "😒":"يستهزء",
    "🙄":"ملل",
    "😕":"عصبية",
    "😃":"يضحك",
    "🌸":"وردة",
    "😓":"حزن",
    "💞":"حب",
    "💗":"حب",
    "😑":"منزعج",
    "💭":"تفكير",
    "😎":"ثقة",
    "💛":"حب",
    "😩":"حزين",
    "💪":"عضلات",
    "👍":"موافق",
    "🙏🏻":"رجاء طلب",
    "😳":"مصدوم",
    "👏🏼":"تصفيق",
    "🎶":"موسيقي",
    "🌚":"صمت",
    "💚":"حب",
    "🙏":"رجاء طلب",
    "💘":"حب",
    "🍃":"سلام",
    "☺":"يضحك",
    "🐸":"ضفدع",
    "😶":"مصدوم",
    "✌️":"مرح",
    "✋🏻":"توقف",
    "😉":"غمزة",
    "🌷":"حب",
    "🙃":"مبتسم",
    "😫":"حزين",
    "😨":"مصدوم",
    "🎼 ":"موسيقي",
    "🍁":"مرح",
    "🍂":"مرح",
    "💟":"حب",
    "😪":"حزن",
    "😆":"يضحك",
    "😣":"استياء",
    "☺️":"حب",
    "😱":"كارثة",
    "😁":"يضحك",
    "😖":"استياء",
    "🏃🏼":"يجري",
    "😡":"غضب",
    "🚶":"يسير",
    "🤕":"مرض",
    "‼️":"تعجب",
    "🕊":"طائر",
    "👌🏻":"احسنت",
    "❣":"حب",
    "🙊":"مصدوم",
    "💃":"سعادة مرح",
    "💃🏼":"سعادة مرح",
    "😜":"مرح",
    "👊":"ضربة",
    "😟":"استياء",
    "💖":"حب",
    "😥":"حزن",
    "🎻":"موسيقي",
    "✒":"يكتب",
    "🚶🏻":"يسير",
    "💎":"الماظ",
    "😷":"وباء مرض",
    "☝":"واحد",
    "🚬":"تدخين",
    "💐" : "ورد",
    "🌞" : "شمس",
    "👆" : "الاول",
    "⚠️" :"تحذير",
    "🤗" : "احتواء",
    "✖️": "غلط",
    "📍"  : "مكان",
    "👸" : "ملكه",
    "👑" : "تاج",
    "✔️" : "صح",
    "💌": "قلب",
    "😲" : "مندهش",
    "💦": "ماء",
    "🚫" : "خطا",
    "👏🏻" : "برافو",
    "🏊" :"يسبح",
    "👍🏻": "تمام",
    "⭕️" :"دائره كبيره",
    "🎷" : "ساكسفون",
    "👋": "تلويح باليد",
    "✌🏼": "علامه النصر",
    "🌝":"مبتسم",
    "➿"  : "عقده مزدوجه",
    "💪🏼" : "قوي",
    "📩":  "تواصل معي",
    "☕️": "قهوه",
    "😧" : "قلق و صدمة",
    "🗨": "رسالة",   
    "❗️" :"تعجب",
    "🙆🏻": "اشاره موافقه",
    "👯" :"اخوات",
    "©" :  "رمز",
    "👵🏽" :"سيده عجوزه",
    "🐣": "كتكوت",  
    "🙌": "تشجيع",
    "🙇": "شخص ينحني",
    "👐🏽":"ايدي مفتوحه",    
    "👌🏽": "بالظبط",
    "⁉️" : "استنكار",
    "⚽️": "كوره",
    "🕶" :"حب",
    "🎈" :"بالون",    
    "🎀":    "ورده",
    "💵":  "فلوس",   
    "😋":  "جائع",
    "😛":  "يغيظ",
    "😠":  "غاضب",
    "✍🏻":  "يكتب",
    "🌾":  "ارز",
    "👣":  "اثر قدمين",
    "❌":"رفض",
    "🍟":"طعام",
    "👬":"صداقة",
    "🐰":"ارنب",
    "☂":"مطر",
    "⚜":"مملكة فرنسا",
    "🐑":"خروف",
    "🗣":"صوت مرتفع",
    "👌🏼":"احسنت",
    "☘":"مرح",
    "😮":"صدمة",
    "😦":"قلق",
    "⭕":"الحق",
    "✏️":"قلم",
    "ℹ":"معلومات",
    "🙍🏻":"رفض",
    "⚪️":"نضارة نقاء",
    "🐤":"حزن",
    "💫":"مرح",
    "💝":"حب",
    "🍔":"طعام",
    "❤︎":"حب",
    "✈️":"سفر",
    "🏃🏻‍♀️":"يسير",
    "🍳":"ذكر",
    "🎤":"مايك غناء",
    "🎾":"كره",
    "🐔":"دجاجة",
    "🙋":"سؤال",
    "📮":"بحر",
    "💉":"دواء",
    "🙏🏼":"رجاء طلب",
    "💂🏿 ":"حارس",
    "🎬":"سينما",
    "♦️":"مرح",
    "💡":"قكرة",
    "‼":"تعجب",
    "👼":"طفل",
    "🔑":"مفتاح",
    "♥️":"حب",
    "🕋":"كعبة",
    "🐓":"دجاجة",
    "💩":"معترض",
    "👽":"فضائي",
    "☔️":"مطر",
    "🍷":"عصير",
    "🌟":"نجمة",
    "☁️":"سحب",
    "👃":"معترض",
    "🌺":"مرح",
    "🔪":"سكينة",
    "♨":"سخونية",
    "👊🏼":"ضرب",
    "✏":"قلم",
    "🚶🏾‍♀️":"يسير",
    "👊":"ضربة",
    "◾️":"وقف",
    "😚":"حب",
    "🔸":"مرح",
    "👎🏻":"لا يعجبني",
    "👊🏽":"ضربة",
    "😙":"حب",
    "🎥":"تصوير",
    "👉":"جذب انتباه",
    "👏🏽":"يصفق",
    "💪🏻":"عضلات",
    "🏴":"اسود",
    "🔥":"حريق",  
    "😬":"عدم الراحة",   
    "👊🏿":"يضرب",    
    "🌿":"ورقه شجره",     
    "✋🏼":"كف ايد",    
    "👐":"ايدي مفتوحه",      
    "☠️":"وجه مرعب",     
    "🎉":"يهنئ",      
    "🔕" :"صامت",
    "😿":"وجه حزين",      
    "☹️":"وجه يائس",     
    "😘" :"حب",     
    "😰" :"خوف و حزن",
    "🌼":"ورده",      
    "💋":  "بوسه",
    "👇":"لاسفل",     
    "❣️":"حب",     
    "🎧":"سماعات",
    "📝":"يكتب",      
    "😇":"دايخ",      
    "😈":"رعب",      
    "🏃":"يجري",      
    "✌🏻":"علامه النصر",    
    "🔫":"يضرب",      
    "❗️":"تعجب",
    "👎":"غير موافق",      
    "🔐":"قفل",      
    "👈":"لليمين",
    "™":"رمز",    
    "🚶🏽":"يتمشي",    
    "😯":"متفاجأ",  
    "✊":"يد مغلقه",    
    "😻":"اعجاب",    
    "🙉" :"قرد",    
    "👧":"طفله صغيره",     
    "🔴":"دائره حمراء",      
    "💪🏽":"قوه",     
    "💤":"ينام",     
    "👀":"ينظر",     
    "✍🏻":"يكتب",  
    "❄️":"تلج",
    "💀":"رعب",   
    "😤":"وجه عابس",      
    "🖋":"قلم",      
    "🎩":"كاب",      
    "☕️":"قهوه",     
    "😹":"ضحك",     
    "💓":"حب",      
    "☄️ ":"نار",     
    "👻":"رعب",
    }

emoticons_to_emoji = {
    ":)" : "🙂",
    ":(" : "🙁",
    "xD" : "😆",
    ":=(": "😭",
    ":'(": "😢",
    ":'‑(": "😢",
    "XD" : "😂",
    ":D" : "🙂",
    "♬" : "موسيقي",
    "♡" : "❤",
    "☻"  : "🙂",
    }

### Functions

In [ ]:
def remove_stop_words(text):
    Text=[i for i in str(text).split() if i not in arabic_stopwords]
    return " ".join(Text)

def ISRI_Stemmer(text):
    #making an object
    stemmer = ISRIStemmer()
    
    #stemming each word
    text = stemmer.stem(text)
    text = stemmer.pre32(text)
    text = stemmer.suf32(text)
    
    return text

def Snowball_stemmer(text):
    text = text.split()
    #making an object
    stemmer = SnowballStemmer("arabic")
    
    #stemming each word
    text=[stemmer.stem(y) for y in text]
    
    return " " .join(text)

def Arabic_Light_Stemmer(text):
    #making an object
    Arabic_Stemmer = ArabicLightStemmer()
    
    #stemming each word
    text=[Arabic_Stemmer.light_stem(y) for y in text.split()]
    
    return " " .join(text)

def normalizeArabic(text):
    text = text.strip()
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    
    #remove repetetions
    text = re.sub("[إأٱآا]", "ا", text)
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('ييي', 'ي')
    text = text.replace('اا', 'ا')

    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)
    
    # Remove longation
    text = re.sub(r'(.)\1+', r"\1\1", text) 
    
    #Strip vowels from a text, include Shadda.
    text = araby.strip_tashkeel(text)
    
    #Strip diacritics from a text, include harakats and small lettres The striped marks are
    text = araby.strip_diacritics(text)
    text=''.join([i for i in text if not i.isdigit()])
    return text

def Removing_non_arabic(text):
    text = re.sub('[A-Za-z]+',' ',text)
    return text

def Removing_numbers(text):
    text=''.join([i for i in text if not i.isdigit()])
    return text

def Removing_punctuations(text):
    ## Remove punctuations
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
    text = text.replace('؛',"", )
    
    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)
    text =  " ".join(text.split())
    return text.strip()

def Removing_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_emoji(string):
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string).strip()

def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text).strip()



def remove_extra_Space(text):
    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)
    return  " ".join(text.split())

def remove_small_sentences(df):
    for i in range(len(df)):
        if len(df.text.iloc[i].split()) < 3:
            df.text.iloc[i] = np.nan
            
            
def replace_emoticon_with_emojis(text):
    for i in range (len(text)):
        message = text.iloc[i]
        seperarate_word = message.split(' ')
        cleaned_data.text.iloc[i] = ""

        for word in seperarate_word:
            text.iloc[i] += emoticons_to_emoji.get(word, word) + " "

def replace_emojis_with_text(text):
    for i in range (len(text)):
            message = text.iloc[i]
            seperarate_word = regex.findall(r'\X', message)
            text.iloc[i] = ""

            for word in seperarate_word:
                if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
                        text.iloc[i] += " " + emojis.get(word, word) + " "
                else:
                        text.iloc[i] +=  emojis.get(word, word) + ""
            
def space_between_emojis(s): 
    return ''.join((' '+c+' ') if c in emoji.UNICODE_EMOJI['en'] else c for c in s)


In [ ]:
cleaned_data = pd.read_excel('NileUniversity_Emotional.xlsx')

In [ ]:
cleaned_data.head()

**drop unnecessary columns**

In [ ]:
cleaned_data.drop('ID', axis=1,inplace=True)

**removing null values**

In [ ]:
cleaned_data.drop(8619, axis=0, inplace = True)

In [ ]:
#Before
cleaned_data.isnull().sum()

In [ ]:
#Ater
cleaned_data.isnull().sum()

**removing duplicated values**

In [ ]:
num_of_duplications = cleaned_data.duplicated().sum()
index = cleaned_data[cleaned_data['text'].duplicated() == True].index
cleaned_data.drop(index, axis = 0, inplace = True)

#resetting the index of the data
cleaned_data.reset_index(drop=True, inplace = True)

In [ ]:
#Before
num_of_duplications

In [ ]:
#Ater
cleaned_data.duplicated().sum()

**Replace emoticons with emojis**

In [ ]:
#cleaned_data.text=cleaned_data.text.apply(lambda text : remove_emoji(text))
replace_emoticon_with_emojis(cleaned_data.text)

In [ ]:
#Before
df.iloc[2].text

In [ ]:
#After
cleaned_data.iloc[2].text

In [ ]:
#we will save this data mapping the emojis into words
cleaned_data.to_csv('cleaned_data_with_emojis.csv', index = False, encoding='utf-8-sig')

**Replace each emoji with a text**

In [ ]:
replace_emojis_with_text(cleaned_data.text)

In [ ]:
cleaned_data.iloc[2].text

**removing arabic stopwords**

In [ ]:
cleaned_data.text=cleaned_data.text.apply(lambda text : remove_stop_words(text))

In [ ]:
#Before
df.text[1]

In [ ]:
#After
cleaned_data.text[1]

**Removal of non-Arabic words**

In [ ]:
cleaned_data.text=cleaned_data.text.apply(lambda text : Removing_non_arabic(text))

In [ ]:
#Before
df.text.iloc[10016]

In [ ]:
#After
cleaned_data.iloc[10016].text

**removing Diacritics**

In [ ]:
cleaned_data.text=cleaned_data.text.apply(lambda text : normalizeArabic(text))

In [ ]:
df.text.iloc[99]

In [ ]:
cleaned_data.text.iloc[99]

**Removal of numbers**

In [ ]:
cleaned_data.text=cleaned_data.text.apply(lambda text : Removing_numbers(text))

In [ ]:
#Before
df.text.iloc[10015]

In [ ]:
cleaned_data.iloc[10015].text

**Removing Hastags**

In [ ]:
for i in range(len(cleaned_data)):
    index = cleaned_data.text.iloc[i].find("#")
    if index != -1:
        print(index)

it seems that the data doesn't contains hastgas

**Removal of Links**

In [ ]:
cleaned_data.text=cleaned_data.text.apply(lambda text : Removing_urls(text))

**Removing Punctuations**

In [ ]:
cleaned_data.text=cleaned_data.text.apply(lambda text : Removing_punctuations(text))

In [ ]:
#Before
df.text.iloc[0]

In [ ]:
#After
cleaned_data.text.iloc[0]

**Reducing words to their roots**

In [ ]:
cleaned_data.text=cleaned_data.text.apply(lambda text : ISRI_Stemmer(text))

In [ ]:
cleaned_data.duplicated().sum()

Note that: duplicated value was found when we remove diacritics and non arabic words and punctuations and this because some rows in the dataset only contains stopwords or it contains one or two words and other reasons so, we have to remove the duplicated values again

In [ ]:
index = cleaned_data[cleaned_data['text'].duplicated() == True].index
cleaned_data.drop(index, axis = 0, inplace = True)

#resetting the index of the data
cleaned_data.reset_index(drop=True, inplace = True)

In [ ]:
cleaned_data.duplicated().sum()

After removing the duplicated values we will remove every row which contains only one or two words berfore preprocessing

In [ ]:
#this function will convert the text which contains one or two words into null value
remove_small_sentences(cleaned_data)

In [ ]:
cleaned_data.isnull().sum()

In [ ]:
cleaned_data.dropna(inplace = True)

In [ ]:
cleaned_data.to_csv('cleaned_data_IRIS.csv', index = False, encoding='utf-8-sig')

## Modeling

In [ ]:
X = cleaned_data.text.values
y = cleaned_data.label.values

In [ ]:
print(X[:5])

In [ ]:
print(y[:5])

**Creating a pipline using tf-idf for words embedding and different models.**

In [ ]:
def train_model(model, data, targets):
    text_clf = Pipeline([('vect',TfidfVectorizer()),
                         ('clf', model)])
    text_clf.fit(data, targets)
    return text_clf

In [ ]:
def get_F1(trained_model,X,y):
    predicted=trained_model.predict(X)
    f1=f1_score(y,predicted, average=None)
    return f1

**Splitting the dataset into the Training set and Test set**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1,shuffle=True, random_state=42)

### Logistic Regression

**Training the Logistic Regression model on the Training set**

In [ ]:
log_reg = train_model(LogisticRegression(random_state = 0), X_train, y_train)

**Predicting new data**

In [ ]:
y_pred=log_reg.predict(['سعيد'])
y_pred

In [ ]:
y_pred=log_reg.predict(['حزن'])
y_pred

In [ ]:
y_pred=log_reg.predict(['مستغرب'])
y_pred

**Predicting the Test set results**

In [ ]:
y_pred=log_reg.predict(X_test)

**Measuring the performance**

In [ ]:
log_reg_accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: ', log_reg_accuracy,'\n')

In [ ]:
f1_Score = get_F1(log_reg,X_test,y_test)
pd.DataFrame(f1_Score, index=df.label.unique(), columns=['F1 score'])

In [ ]:
##Classification Report
print(classification_report(y_test, y_pred))

### Decision Tree

**Training the Decision Tree model on the Training set**

In [ ]:
DT = train_model(DecisionTreeClassifier(random_state = 0), X_train, y_train)

**Predicting the Test set results**

In [ ]:
y_pred=DT.predict(X_test)

**Measuring the performance**

In [ ]:
DT_accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: ', DT_accuracy,'\n')

In [ ]:
f1_Score = get_F1(DT, X_test ,y_test)
pd.DataFrame(f1_Score, index=df.label.unique(), columns=['F1 score'])

In [ ]:
##Classification Report
print(classification_report(y_test, y_pred))

### Support Vector Machine

**Training the Support Vector Machine model on the Training set**

In [ ]:
SVM = train_model(SVC(random_state = 0), X_train, y_train)

**Predicting the Test set results**

In [ ]:
y_pred=SVM.predict(X_test)

**Measuring the performance**

In [ ]:
SVM_accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: ', SVM_accuracy,'\n')

In [ ]:
f1_Score = get_F1(SVM,X_test,y_test)
pd.DataFrame(f1_Score, index=df.label.unique(), columns=['F1 score'])

In [ ]:
##Classification Report
print(classification_report(y_test, y_pred))

### Random Forest

**Training the Random Forest model on the Training set**

In [ ]:
RF = train_model(RandomForestClassifier(random_state = 0), X_train, y_train)

**Predicting the Test set results**

In [ ]:
y_pred=RF.predict(X_test)

**Measuring the performance**

In [ ]:
RF_accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: ', RF_accuracy,'\n')

In [ ]:
f1_Score = get_F1(RF, X_test, y_test)
pd.DataFrame(f1_Score, index=df.label.unique(), columns=['F1 score'])

In [ ]:
##Classification Report
print(classification_report(y_test, y_pred))

## Results

In [ ]:
models = pd.DataFrame({
    'Model': ['Logistic Regression', 'Decision Tree','Support Vector Machine','Random Forest'],
    'Accuracy': [log_reg_accuracy.round(2), DT_accuracy.round(2), SVM_accuracy.round(2), RF_accuracy.round(2)]})

models.sort_values(by='Accuracy', ascending=False).reset_index().drop(['index'], axis=1)

Note that TF-IDF vectoriser is used in this experiment